# 1. Downloading data, clean and preprocess



In [1]:
import os
import re
import nltk
import numpy as np
import pandas as pd
from zipfile import ZipFile
import zipfile
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package punkt to /home/bptran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/bptran/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/bptran/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/bptran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/bptran/anaconda3/envs/sota/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install torchinfo
import torchinfo

## 1.1 Loading data 

In [3]:
# Fetch the data
path = "data_full3.zip"

df = pd.DataFrame()
with zipfile.ZipFile(path) as z:
  for name in z.namelist():
    if name.endswith(".csv"):
      print(f'Loading data from {name}...')
      x = pd.read_csv(z.open(name))
      print(f'Loading completed from {name}...')
      df = pd.concat([df, x[['genre','plot']]],axis=0,ignore_index=True)
  print("Dataframe (df) ready to be used!")

df

Loading data from content/gdrive/MyDrive/CS5242_Project_Data/action/action_films.csv...
Loading completed from content/gdrive/MyDrive/CS5242_Project_Data/action/action_films.csv...
Loading data from content/gdrive/MyDrive/CS5242_Project_Data/adventure/adventure_films.csv...
Loading completed from content/gdrive/MyDrive/CS5242_Project_Data/adventure/adventure_films.csv...
Loading data from content/gdrive/MyDrive/CS5242_Project_Data/comedy/comedy_films.csv...
Loading completed from content/gdrive/MyDrive/CS5242_Project_Data/comedy/comedy_films.csv...
Loading data from content/gdrive/MyDrive/CS5242_Project_Data/drama/drama_films.csv...
Loading completed from content/gdrive/MyDrive/CS5242_Project_Data/drama/drama_films.csv...
Loading data from content/gdrive/MyDrive/CS5242_Project_Data/horror/horror_films.csv...
Loading completed from content/gdrive/MyDrive/CS5242_Project_Data/horror/horror_films.csv...
Loading data from content/gdrive/MyDrive/CS5242_Project_Data/romance/romance_films.csv.

,genre,plot
0,action,frank vega is a decorated vietnam war veteran ...
1,action,journalist matt nashs dylan walsh investigatio...
2,action,in the midtolate 1960s three young men leave t...
3,action,po sing the youngest son of chinese triad boss...
4,action,clay santell audie murphy has his horse stolen...
...,...,...
2677,crime,a mexican newspaperman wages a oneman war agai...
2678,crime,larry crain peter cookson a medical student on...
2679,crime,a texas ranger samantha payne opens up a fifte...
2680,crime,billy dempsey is a well dressed bank robber wh...


In [353]:
# # Fetch the data, clip to 200 each for experiments
# path = "data_full3.zip"

# df = pd.DataFrame()
# with zipfile.ZipFile(path) as z:
#   for name in z.namelist():
#     if name.endswith(".csv"):
#       print(f'Loading data from {name}...')
#       x = pd.read_csv(z.open(name)).head(200)
#       print(f'Loading completed from {name}...')
#       df = pd.concat([df, x[['genre','plot']]],axis=0,ignore_index=True)
#   print("Dataframe (df) ready to be used!")

# df

# # Clip to 1000 length of each plot
# df.loc[:, 'plot'] = df['plot'].apply(lambda x: ' '.join(x.split()[0:1000]))

In [4]:
df.groupby('genre').count()

,plot
genre,
action,299
adventure,286
comedy,276
crime,271
drama,279
fantasy,209
historical,249
horror,342
romance,160


## 1.2. Clean data

In [5]:
import re

from nltk import word_tokenize, PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')


class SentCleaner:
    def __init__(self, sent, conf):
        self.sent = sent
        self.sent_tokenized = None
        self.conf = conf

    def lower_case(self):
        if self.conf['lower']:
            self.sent = self.sent.lower()
        return self

    def tokenize(self):
        if self.conf['token']:
            self.sent_tokenized = word_tokenize(self.sent)
        return self

    def remove_stopwords(self, stop_words):
        if self.conf['remove_stop']:
            self.sent_tokenized = [word for word in self.sent_tokenized if word not in stop_words]
        return self

    def remove_punct(self):
        if self.conf['remove_punc']:
            self.sent_tokenized = [word for word in self.sent_tokenized if re.search('[a-z]', word)]
        return self

    def stem_words(self):
        if self.conf['stem']:
            stemmer = PorterStemmer()
            self.sent_tokenized = [stemmer.stem(word) for word in self.sent_tokenized]
        return self

    def remove_escapes(self):
        if self.conf['remove_esc']:
            stripped = [word.replace('\n', '') for word in self.sent_tokenized]
            self.sent_tokenized = [word for word in stripped if word != '']
        return self

    def clean_sent(self):
        self.lower_case() \
            .tokenize() \
            .remove_punct() \
            .remove_escapes() \
            .stem_words() \
            .remove_stopwords(stopwords.words('english'))

        return ' '.join(self.sent_tokenized)
    
    
# Sentence cleaner conf
sent_cleaner_conf = dict()
sent_cleaner_conf['token']  = True
sent_cleaner_conf['lower']  = True
sent_cleaner_conf['encode'] = True
sent_cleaner_conf['remove_stop'] = False
sent_cleaner_conf['remove_punc'] = False
sent_cleaner_conf['remove_esc'] = False
sent_cleaner_conf['stem'] = True

# Transform plots into vectors using lookup
#df.loc[:, 'plot'] = df['plot'].apply(lambda x: SentCleaner(x, sent_cleaner_conf).clean_sent())

[nltk_data] Downloading package punkt to /home/bptran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/bptran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df

,genre,plot
0,action,frank vega is a decorated vietnam war veteran ...
1,action,journalist matt nashs dylan walsh investigatio...
2,action,in the midtolate 1960s three young men leave t...
3,action,po sing the youngest son of chinese triad boss...
4,action,clay santell audie murphy has his horse stolen...
...,...,...
2677,crime,a mexican newspaperman wages a oneman war agai...
2678,crime,larry crain peter cookson a medical student on...
2679,crime,a texas ranger samantha payne opens up a fifte...
2680,crime,billy dempsey is a well dressed bank robber wh...


In [7]:
#Commence data cleaning, split data into X and Y
plot_list, genre_list = df['plot'] , df['genre']

#Load, pre-process and clean data before splitting into train and test set
lemma = WordNetLemmatizer()


## 1.3. Tokenization, embeddings

In [8]:
def tokenize(texts):
    """Tokenize texts, build vocabulary and find maximum sentence length.
    
    Args:
        texts (List[str]): List of text data
    
    Returns:
        tokenized_texts (List[List[str]]): List of list of tokens
        word2idx (Dict): Vocabulary built from the corpus
        max_len (int): Maximum sentence length
    """

    max_len = 0
    tokenized_texts = []
    word2idx = {}

    # Add <pad> and <unk> tokens to the vocabulary
    word2idx['<pad>'] = 0
    word2idx['<unk>'] = 1

    # Building our vocab from the corpus starting from index 2
    idx = 2
    for sent in texts:
        tokenized_sent = word_tokenize(sent)

        # Add `tokenized_sent` to `tokenized_texts`
        tokenized_texts.append(tokenized_sent)

        # Add new token to `word2idx`
        for token in tokenized_sent:
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1

        # Update `max_len`
        max_len = max(max_len, len(tokenized_sent))

    return tokenized_texts, word2idx, max_len

def encode(tokenized_texts, word2idx, max_len):
    """Pad each sentence to the maximum sentence length and encode tokens to
    their index in the vocabulary.

    Returns:
        input_ids (np.array): Array of token indexes in the vocabulary with
            shape (N, max_len). It will the input of our CNN model.
    """

    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Pad sentences to max_len
        padd_tokenized_sent = tokenized_sent + ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        input_id = [word2idx.get(token) for token in padd_tokenized_sent]
        input_ids.append(input_id)
    
    return np.array(input_ids)

def encode_no_pad(tokenized_texts, word2idx):
    """Pad each sentence to the maximum sentence length and encode tokens to
    their index in the vocabulary.

    Returns:
        input_ids (np.array): Array of token indexes in the vocabulary with
            shape (N, max_len). It will the input of our CNN model.
    """

    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Pad sentences to max_len

        # Encode tokens to input_ids
        input_id = [word2idx.get(token) for token in tokenized_sent]
        input_ids.append(input_id)
    
    return input_ids

In [9]:
from tqdm import tqdm

def load_pretrained_vectors(word2idx, fname):
    """Load pretrained vectors and create embedding layers.
    
    Args:
        word2idx (Dict): Vocabulary built from the corpus
        fname (str): Path to pretrained vector file

    Returns:
        embeddings (np.array): Embedding matrix with shape (N, d) where N is
            the size of word2idx and d is embedding dimension
    """

    print("Loading pretrained vectors...")
    fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    if "glove" in fname:
        d = len(fin.readline().split()) - 1
    elif "crawl" in fname:
        n, d = map(int, fin.readline().split())

    # Initilize random embeddings
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
    embeddings[word2idx['<pad>']] = np.zeros((d,))

    # Load pretrained vectors
    count = 0
    for line in tqdm(fin):
        tokens = line.rstrip().split(' ')
        word = tokens[0]
        if word in word2idx:
            count += 1
            embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)

    print(f"There are {count} / {len(word2idx)} pretrained vectors found.")

    return embeddings

In [10]:
# Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_texts, word2idx, max_len = tokenize(plot_list)

# Load pretrained vectors, please put 'glove.6B.300d.txt' in current folder to be run.
# 'glove.6B.300d.txt' is downloaded at https://www.kaggle.com/datasets/thanakomsn/glove6b300dtxt
embeddings = load_pretrained_vectors(word2idx, "glove.6B.300d.txt")
embeddings = torch.tensor(embeddings)

Tokenizing...

Loading pretrained vectors...


399999it [00:08, 47237.62it/s]


There are 34162 / 41299 pretrained vectors found.


## 1.4. Preparing dataset and dataloader

In [11]:
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler, Dataset,
                              SequentialSampler)

from typing import List

class FilmGenres(Dataset):
    def __init__(self, inputs: List, labels: np.ndarray):
        '''
        train: True if dataset for training, otherwise False is for testing
        zip_file_path is path containing the plot films
        '''
        self.list_of_plot = inputs
        self.array_of_labels = labels


    def __len__(self):
        return len(self.list_of_plot)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        if idx >= len(self):
            raise IndexError

        sample = torch.tensor(self.list_of_plot[idx]), torch.tensor(self.array_of_labels[idx])

        return sample

def data_loader(train_inputs, val_inputs, train_labels, val_labels,batch_size=1):
    """Convert train and validation sets to torch.Tensors and load them to
    DataLoader.
    """


    # Create DataLoader for training data
    train_data = FilmGenres(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create DataLoader for validation data
    val_data = FilmGenres(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader

In [12]:
from sklearn.model_selection import train_test_split

# Train Test Split

# 1. Noting here, if use batch_size > 1, then must use encode
# 2. If use batch_size = 1, then can use encode or encode_no_pad

# train_inputs is list of list of integer number where number is encoded of token, and train_labels is numpy array
# encode tokens

input_ids = encode_no_pad(tokenized_texts, word2idx) # For batch_size = 1
#input_ids = encode(tokenized_texts, word2idx, max_len) # For batch_size > 1

#Transformation for Y#
#create dictionary to retain information on different classes
#action, adventure, comedy, crime, drama, fantasy, historical, horror, romance, scifi
y_info = {'action': 0, 'adventure':1,'comedy':2,'crime':3, 'drama':4,'fantasy':5,'historical':6,'horror':7,'romance':8,'scifi':9}
Y = genre_list.apply(lambda y: y_info[y.lower()])

#Transformation for Y, one hot encode for multiclass
Y = Y.to_numpy() #In order to use reshape below

train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, Y, test_size=0.1, random_state=42)

In [13]:
# Load data to PyTorch DataLoader
train_dataloader, val_dataloader = data_loader(train_inputs, val_inputs, train_labels, val_labels, batch_size=1)

# 2. Common train / test procedure for training

In [14]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3060 Ti


In [15]:
import random
import time
from tqdm import tqdm

# Specify loss function
loss_fn = nn.CrossEntropyLoss() 


def set_seed(seed_value=42):
    """Set seed for reproducibility."""

    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, optimizer, train_dataloader, val_dataloader=None, epochs=10):
    """Train the CNN model."""
    
    # Tracking best validation accuracy
    best_accuracy = 0

    # Start training loop
    print("Start training...\n")
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
    print("-"*60)

    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================

        # Tracking time and loss
        t0_epoch = time.time()
        running_loss = 0

        # Put the model into the training mode
        model.train()

        for step, batch in enumerate(tqdm(train_dataloader)):
            # Load batch to GPU
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)
            # Perform a forward pass. This will return logits.
            model_outputs = model(b_input_ids) #added by myself

            # compute loss
            loss = loss_fn(model_outputs,b_labels) #added by myself

            # Zero out any previously calculated gradients
            optimizer.zero_grad() #added by myself

            # Perform a backward pass to calculate gradients
            loss.backward() #added by myself

            # Update parameters
            optimizer.step()

            # Compute loss and accumulate the loss values
            running_loss += loss.item()
            

        # Calculate the average loss over the entire training data
        avg_train_loss = running_loss / len(train_dataloader)

        # =======================================
        #               Evaluation
        # =======================================
        if val_dataloader is not None:
            # After the completion of each training epoch, measure the model's
            # performance on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Track the best accuracy
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }, f'{PATH}/cnn.pt')

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            
            with open(f'{PATH}/log.txt', 'a') as f:
                f.write(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}\n")
            
    print("\n")
    print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")

def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's
    performance on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled
    # during the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

# 2. CNN

In [133]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_NLP(nn.Module):
    """An 1D Convulational Neural Network for Sentence Classification."""
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=2,
                 dropout=0.5):
        """
        The constructor for CNN_NLP class.

        Args:
            pretrained_embedding (torch.Tensor): Pretrained embeddings with
                shape (vocab_size, embed_dim)
            freeze_embedding (bool): Set to False to fine-tune pretraiend
                vectors. Default: False
            vocab_size (int): Need to be specified when not pretrained word
                embeddings are not used.
            embed_dim (int): Dimension of word vectors. Need to be specified
                when pretrained word embeddings are not used. Default: 300
            filter_sizes (List[int]): List of filter sizes. Default: [3, 4, 5]
            num_filters (List[int]): List of number of filters, has the same
                length as `filter_sizes`. Default: [100, 100, 100]
            n_classes (int): Number of classes. Default: 2
            dropout (float): Dropout rate. Default: 0.5
        """

        super(CNN_NLP, self).__init__()
        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        """Perform a forward pass through the network.

        Args:
            input_ids (torch.Tensor): A tensor of token ids with shape
                (batch_size, max_sent_length)

        Returns:
            logits (torch.Tensor): Output logits with shape (batch_size,
                n_classes)
        """

        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.embedding(input_ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]
        
        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        
        # Compute logits. Output shape: (b, n_classes)
        logits = self.fc(x_fc)

        return logits

In [175]:
import torch.optim as optim

def initilize_cnn_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    filter_sizes=[3, 4, 5],
                    num_filters=[100, 100, 100],
                    num_classes=2,
                    dropout=0.5,
                    learning_rate=0.01):
    """Instantiate a CNN model and an optimizer."""

    assert (len(filter_sizes) == len(num_filters)), "filter_sizes and \
    num_filters need to be of the same length."

    # Instantiate CNN model
    cnn_model = CNN_NLP(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        filter_sizes=filter_sizes,
                        num_filters=num_filters,
                        num_classes=num_classes,
                        dropout=dropout)
    
    # Send model to `device` (GPU/CPU)
    cnn_model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(cnn_model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    # For cnn4
#     optimizer = optim.Adam(cnn_model.parameters(),
#                                lr=learning_rate)

    return cnn_model, optimizer

In [66]:
# 1. CNN model: default params, no pretrained word embeddings.
# Accurcy for this model is 43.12%
set_seed(42)
filter_sizes = [3, 3, 3]
num_filters = [64, 128, 256]
PATH = "cnn1"
os.makedirs(PATH, exist_ok=True)

model, optimizer = initilize_cnn_model(pretrained_embedding=None,
                                            freeze_embedding=True,
                                            filter_sizes=filter_sizes,
                                            num_filters=num_filters,
                                            num_classes=len(y_info),
                                            vocab_size = len(word2idx),
                                            dropout=0,
                                            learning_rate=0.01)
train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:14<00:00, 161.12it/s]


   1    |   2.293282   |  2.268148  |   19.33   |   18.70  


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:15<00:00, 157.01it/s]


   2    |   2.220484   |  2.253192  |   17.10   |   15.54  


...

  98    |   0.005268   |  1.886382  |   39.41   |   15.62  


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:15<00:00, 159.38it/s]


  99    |   0.005042   |  1.891585  |   37.92   |   15.32  


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:14<00:00, 164.07it/s]


  100   |   0.004969   |  1.899401  |   38.66   |   14.91  


Training complete! Best accuracy: 43.12%.


In [69]:
# 2. CNN model: golve.6b.100d pretrained word vectors are fine-tuned during training.
# Accurcy for this model is 57.99%
PATH = "cnn2"
os.makedirs(PATH, exist_ok=True)

set_seed(42)
model, optimizer = initilize_cnn_model(pretrained_embedding=embeddings,
                                            freeze_embedding=True,
                                            filter_sizes=filter_sizes,
                                            num_filters=num_filters,
                                            num_classes=len(y_info),
                                            vocab_size = len(word2idx),
                                            dropout=0,
                                            learning_rate=0.01)
train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 691.41it/s]


   1    |   2.262419   |  2.215146  |   19.70   |   4.08   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 651.31it/s]


   2    |   2.132618   |  2.128111  |   32.71   |   5.27   


...

  98    |   0.003786   |  1.686759  |   55.76   |   3.84   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 633.05it/s]


  99    |   0.003709   |  1.701209  |   57.25   |   4.14   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:04<00:00, 532.88it/s]


  100   |   0.003569   |  1.700915  |   56.88   |   4.70   


Training complete! Best accuracy: 57.99%.


In [71]:
# 3. Higher learning rate [0.05, 0.1], with pre-trained embeddings
# Accurcy for this model is 
# 0.05 : 57.99%
# 0.1: 58.36%
PATH = "cnn3"
os.makedirs(PATH, exist_ok=True)

set_seed(42)
model, optimizer = initilize_cnn_model(pretrained_embedding=embeddings,
                                            freeze_embedding=True,
                                            filter_sizes=filter_sizes,
                                            num_filters=num_filters,
                                            num_classes=len(y_info),
                                            vocab_size = len(word2idx),
                                            dropout=0,
                                            learning_rate=0.1)

# To save time, loading only model from 'cnn2'
checkpoint = torch.load('cnn2/cnn_begin_pretrained.pt')
model.load_state_dict(checkpoint['model_state_dict'])

train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 646.12it/s]


   1    |   0.007076   |  1.746131  |   58.36   |   4.72   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 686.45it/s]


   2    |   0.006998   |  1.770568  |   57.62   |   3.69   


...

  98    |   0.007198   |  2.262463  |   56.51   |   3.80   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 638.90it/s]


  99    |   0.007443   |  2.255573  |   56.51   |   3.94   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 619.03it/s]


  100   |   0.007017   |  2.253099  |   56.51   |   4.10   


Training complete! Best accuracy: 58.36%.


In [77]:
# 4. Using Adam, 
# Best accuracy 43.87%, it is too bad. Stop here
PATH = "cnn4"
os.makedirs(PATH, exist_ok=True)

set_seed(42)
model, optimizer = initilize_cnn_model(pretrained_embedding=embeddings,
                                            freeze_embedding=True,
                                            filter_sizes=filter_sizes,
                                            num_filters=num_filters,
                                            num_classes=len(y_info),
                                            vocab_size = len(word2idx),
                                            dropout=0,
                                            learning_rate=0.01)

# To save time, loading only model from 'cnn2'
checkpoint = torch.load('cnn2/cnn_begin_pretrained.pt')
model.load_state_dict(checkpoint['model_state_dict'])

train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:04<00:00, 565.59it/s]


   1    |  27.018479   | 33.889943  |   22.68   |   5.25   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 612.49it/s]


   2    |  31.284025   | 42.353149  |   36.43   |   4.92   


...

  98    |  18.826278   | 1559.134138 |   40.15   |   3.90   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:04<00:00, 587.21it/s]


  99    |  13.397267   | 1650.620328 |   35.32   |   4.28   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 693.07it/s]


  100   |  12.579413   | 1568.379871 |   36.43   |   3.65   


Training complete! Best accuracy: 43.87%.


In [81]:
# 5. Change kernel and filter size
# Best accuracy 56.51%. We can keep previous settings
PATH = "cnn5"
os.makedirs(PATH, exist_ok=True)
filter_sizes = [3, 4, 5]
num_filters = [128, 256, 512]

set_seed(42)
model, optimizer = initilize_cnn_model(pretrained_embedding=embeddings,
                                            freeze_embedding=True,
                                            filter_sizes=filter_sizes,
                                            num_filters=num_filters,
                                            num_classes=len(y_info),
                                            vocab_size = len(word2idx),
                                            dropout=0,
                                            learning_rate=0.01)


train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:04<00:00, 493.41it/s]


   1    |   2.248771   |  2.175104  |   22.30   |   6.02   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:04<00:00, 548.68it/s]


   2    |   2.045843   |  2.073203  |   30.11   |   5.76   


...

  98    |   0.002498   |  1.558725  |   55.39   |   4.39   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:04<00:00, 598.92it/s]


  99    |   0.002313   |  1.560645  |   55.39   |   4.21   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 612.28it/s]


  100   |   0.002431   |  1.557722  |   55.76   |   4.12   


Training complete! Best accuracy: 56.51%.


In [83]:
# 6. Adding dropout
# best accuracy 58.36%

PATH = "cnn6"
os.makedirs(PATH, exist_ok=True)
filter_sizes = [3, 3, 3]
num_filters = [64, 128, 256]

set_seed(42)
model, optimizer = initilize_cnn_model(pretrained_embedding=embeddings,
                                            freeze_embedding=True,
                                            filter_sizes=filter_sizes,
                                            num_filters=num_filters,
                                            num_classes=len(y_info),
                                            vocab_size = len(word2idx),
                                            dropout=0.5,
                                            learning_rate=0.01)

# To save time, loading only model from 'cnn2'
checkpoint = torch.load('cnn2/cnn_begin_pretrained.pt')
model.load_state_dict(checkpoint['model_state_dict'])

train(model, optimizer, train_dataloader, val_dataloader, epochs=100)


Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 651.71it/s]


   1    |   0.003874   |  1.691713  |   57.62   |   4.00   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 609.75it/s]


   2    |   0.003915   |  1.692497  |   57.62   |   4.13   


...

  98    |   0.001823   |  1.916051  |   56.13   |   4.17   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 643.78it/s]


  99    |   0.001799   |  1.922180  |   56.88   |   3.92   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 608.89it/s]


  100   |   0.001779   |  1.920510  |   56.51   |   4.15   


Training complete! Best accuracy: 58.36%.


In [85]:
# 7. Using embeddings but also training embeddings. Using dropout=0.5 for best performance
# Accuracy 58.36%

PATH = "cnn7"
os.makedirs(PATH, exist_ok=True)
filter_sizes = [3, 3, 3]
num_filters = [64, 128, 256]

set_seed(42)
model, optimizer = initilize_cnn_model(pretrained_embedding=embeddings,
                                            freeze_embedding=False,
                                            filter_sizes=filter_sizes,
                                            num_filters=num_filters,
                                            num_classes=len(y_info),
                                            vocab_size = len(word2idx),
                                            dropout=0.5,
                                            learning_rate=0.01)

# To save time, loading only model from 'cnn6'
checkpoint = torch.load('cnn6/cnn.pt')
model.load_state_dict(checkpoint['model_state_dict'])

train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:28<00:00, 83.26it/s]


   1    |   0.002889   |  1.751330  |   57.99   |   29.68  


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:25<00:00, 95.16it/s]


   2    |   0.002948   |  1.752914  |   57.99   |   25.62  


...

  98    |   0.001791   |  1.936126  |   55.76   |   23.57  


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:23<00:00, 103.36it/s]


  99    |   0.001779   |  1.941386  |   56.88   |   23.50  


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:23<00:00, 103.31it/s]


  100   |   0.001759   |  1.940019  |   56.51   |   23.52  


Training complete! Best accuracy: 58.36%.


In [100]:
# 8. Clip to 200 to make balance of dataset. Using pretrained, dropout = 0.5 and free training embeddings

# Accuracy 57.14%

PATH = "cnn8"
os.makedirs(PATH, exist_ok=True)
filter_sizes = [3, 3, 3]
num_filters = [64, 128, 256]

set_seed(42)
model, optimizer = initilize_cnn_model(pretrained_embedding=embeddings,
                                            freeze_embedding=True,
                                            filter_sizes=filter_sizes,
                                            num_filters=num_filters,
                                            num_classes=len(y_info),
                                            vocab_size = len(word2idx),
                                            dropout=0.1,
                                            learning_rate=0.005)

# To save time, loading only model from 'cnn6'
#checkpoint = torch.load('cnn6/cnn.pt')
#model.load_state_dict(checkpoint['model_state_dict'])

train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1764/1764 [00:03<00:00, 584.55it/s]


   1    |   2.299594   |  2.286966  |   13.78   |   3.91   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1764/1764 [00:02<00:00, 600.81it/s]


   2    |   2.238713   |  2.258997  |   28.57   |   3.82   


...

  98    |   0.074922   |  1.361479  |   56.12   |   2.64   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1764/1764 [00:02<00:00, 695.25it/s]


  99    |   0.071717   |  1.363690  |   55.10   |   2.66   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1764/1764 [00:02<00:00, 722.89it/s]


  100   |   0.068592   |  1.356034  |   56.12   |   2.57   


Training complete! Best accuracy: 57.14%.


In [103]:
# 9. Clip to 200 to make balance of dataset. Using pretrained, dropout = 0.5 and free training embeddings.
# Using batch

# Accuracy 56.63%

PATH = "cnn9"
os.makedirs(PATH, exist_ok=True)
filter_sizes = [3, 3, 3]
num_filters = [64, 128, 256]

set_seed(42)
model, optimizer = initilize_cnn_model(pretrained_embedding=embeddings,
                                            freeze_embedding=True,
                                            filter_sizes=filter_sizes,
                                            num_filters=num_filters,
                                            num_classes=len(y_info),
                                            vocab_size = len(word2idx),
                                            dropout=0.1,
                                            learning_rate=0.005)

# To save time, loading only model from 'cnn6'
#checkpoint = torch.load('cnn6/cnn.pt')
#model.load_state_dict(checkpoint['model_state_dict'])

train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 441/441 [00:03<00:00, 138.42it/s]


   1    |   2.305943   |  2.294619  |   8.67    |   3.53   


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 441/441 [00:02<00:00, 165.25it/s]


   2    |   2.267723   |  2.279502  |   17.86   |   3.52   


...

  98    |   0.361667   |  1.385420  |   56.12   |   2.99   


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 441/441 [00:02<00:00, 163.73it/s]


  99    |   0.352450   |  1.381488  |   56.12   |   2.89   


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 441/441 [00:02<00:00, 160.47it/s]


  100   |   0.343705   |  1.379291  |   56.63   |   2.96   


Training complete! Best accuracy: 56.63%.


In [139]:
# 10. Clip to 200 to make balance of dataset. Using pretrained, dropout = 0.5 and free training embeddings.
# Clip length to 1000

# Accuracy 58.16%

PATH = "cnn10"
os.makedirs(PATH, exist_ok=True)
filter_sizes = [3, 3, 3]
num_filters = [64, 128, 256]

set_seed(42)
model, optimizer = initilize_cnn_model(pretrained_embedding=embeddings,
                                            freeze_embedding=True,
                                            filter_sizes=filter_sizes,
                                            num_filters=num_filters,
                                            num_classes=len(y_info),
                                            vocab_size = len(word2idx),
                                            dropout=0.1,
                                            learning_rate=0.005)

# To save time, loading only model from 'cnn6'
#checkpoint = torch.load('cnn6/cnn.pt')
#model.load_state_dict(checkpoint['model_state_dict'])

train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1764/1764 [00:03<00:00, 483.94it/s]


   1    |   2.299335   |  2.286443  |   13.78   |   4.44   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1764/1764 [00:02<00:00, 619.92it/s]


   2    |   2.238577   |  2.259026  |   27.55   |   3.65   


...

  98    |   0.074784   |  1.366038  |   55.10   |   2.85   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1764/1764 [00:02<00:00, 599.89it/s]


  99    |   0.071581   |  1.367566  |   55.61   |   3.07   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1764/1764 [00:02<00:00, 678.03it/s]


  100   |   0.068440   |  1.361240  |   55.10   |   2.73   


Training complete! Best accuracy: 58.16%.


In [350]:
# 2. CNN model: golve.6b.100d pretrained word vectors are fine-tuned during training.
# Accurcy for this model is 50.56% all True, 51.65% no stop and escp., remove some is 51.67%i f
PATH = "cnn22"
os.makedirs(PATH, exist_ok=True)

set_seed(42)
model, optimizer = initilize_cnn_model(pretrained_embedding=embeddings,
                                            freeze_embedding=True,
                                            filter_sizes=filter_sizes,
                                            num_filters=num_filters,
                                            num_classes=len(y_info),
                                            vocab_size = len(word2idx),
                                            dropout=0,
                                            learning_rate=0.01)
train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 615.31it/s]


   1    |   2.272877   |  2.236840  |   18.96   |   4.65   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 625.28it/s]


   2    |   2.160587   |  2.165110  |   32.71   |   4.59   


...

  37    |   0.266233   |  1.450174  |   49.44   |   3.74   


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:03<00:00, 617.68it/s]


  38    |   0.248400   |  1.438226  |   50.19   |   4.07   


 81%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                    | 1948/2413 [00:03<00:00, 628.10it/s]


KeyboardInterrupt: 

In [381]:
from torchinfo import summary
summary(model, (1, 100), batch_dim=None, dtypes = [torch.int])

Layer (type:depth-idx)                   Output Shape              Param #
LSTMClassification                       [1, 10]                   --
├─Embedding: 1-1                         [1, 100, 300]             (12,389,700)
├─LSTM: 1-2                              [1, 100, 300]             722,400
├─Linear: 1-3                            [1, 1, 150]               45,150
├─Linear: 1-4                            [1, 1, 10]                1,510
Total params: 13,158,760
Trainable params: 769,060
Non-trainable params: 12,389,700
Total mult-adds (M): 84.68
Input size (MB): 0.00
Forward/backward pass size (MB): 0.48
Params size (MB): 52.64
Estimated Total Size (MB): 53.12

In [ ]:
# just a bit analysis


# 2. RNN

In [389]:
class RNNClassification(nn.Module):

    def __init__(self, vocab_size = None,
                 embed_dim = 128,
                 pretrained_embedding = None,
                 freeze_embedding = True,
                 num_classes = 10):
        super(RNNClassification, self).__init__()

        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        
        self.layer1 = self.embedding
        self.layer2 = nn.RNN(       embed_dim , embed_dim*2  , batch_first=True)
        self.layer3 = nn.RNN(       embed_dim*2 , embed_dim*2  , batch_first=True)

        self.layer4 = nn.Linear(    embed_dim*2 , embed_dim  )
        self.layer5 = nn.Linear(    embed_dim , num_classes)

        
    def forward(self, word_seq ):
        g_seq               =   self.layer1( word_seq )
        h_seq, h_final     =   self.layer2( g_seq  )
        h_seq, h_final     =   self.layer3( h_seq  )

        linear1 = F.relu(self.layer4(h_final))
        score_seq           =   self.layer5( linear1 )
        score_seq = score_seq.squeeze(1) # Unsequeeze the seq dimension
        return score_seq 

In [427]:
def initilize_rnn_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    num_classes=2,
                    learning_rate=0.01):
    """Instantiate a RNN model and an optimizer."""

    # Instantiate RNN model
    rnn_model = RNNClassification(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        num_classes=num_classes)
    
    # Send model to `device` (GPU/CPU)
    rnn_model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(rnn_model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return rnn_model, optimizer

In [428]:
# Best 34.57%
set_seed(42)
PATH = "rnn1"
os.makedirs(PATH, exist_ok=True)
rnn_model, optimizer = initilize_rnn_model(pretrained_embedding=embeddings.float(),
                                            freeze_embedding=True,
                                            vocab_size=len(word2idx),
                                            embed_dim=300,
                                            num_classes=len(y_info),
                                            learning_rate=0.05)
train(rnn_model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:33<00:00, 72.58it/s]


   1    |   2.260403   |  2.237740  |   16.36   |   35.13  


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:32<00:00, 73.93it/s]


   2    |   2.104199   |  2.190195  |   17.84   |   34.52  


...

  98    |   0.754871   |  4.796848  |   25.28   |   33.56  


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:32<00:00, 74.92it/s]


  99    |   0.642498   |  5.302235  |   27.14   |   33.56  


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:32<00:00, 74.90it/s]


  100   |   0.551917   |  5.188927  |   23.42   |   33.56  


Training complete! Best accuracy: 34.57%.


In [392]:
from torchinfo import summary
summary(rnn_model, (1, 100), batch_dim=None, dtypes = [torch.int])

Layer (type:depth-idx)                   Output Shape              Param #
RNNClassification                        [1, 10]                   --
├─Embedding: 1-1                         [1, 100, 300]             (12,389,700)
├─RNN: 1-2                               [1, 100, 600]             541,200
├─RNN: 1-3                               [1, 100, 600]             721,200
├─Linear: 1-4                            [1, 1, 300]               180,300
├─Linear: 1-5                            [1, 1, 10]                3,010
Total params: 13,835,410
Trainable params: 1,445,710
Non-trainable params: 12,389,700
Total mult-adds (M): 138.81
Input size (MB): 0.00
Forward/backward pass size (MB): 1.20
Params size (MB): 55.34
Estimated Total Size (MB): 56.54

# 3. LSTM

In [394]:
class LSTMClassification(nn.Module):

    def __init__(self, vocab_size = None,
                 embed_dim = 128,
                 pretrained_embedding = None,
                 freeze_embedding = True,
                 num_classes = 10):
        super(LSTMClassification, self).__init__()

        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        
        self.layer1 = self.embedding
        self.layer2 = nn.LSTM(       embed_dim , embed_dim *2 , batch_first=True)
        self.layer3 = nn.LSTM(       embed_dim *2, embed_dim *2 , batch_first=True)

        self.layer4 = nn.Linear(    embed_dim *2, embed_dim   )
        self.layer5 = nn.Linear(    embed_dim, num_classes)

        
    def forward(self, word_seq ):
        g_seq               =   self.layer1( word_seq )
        h_seq, (h_final,  c_final)     =   self.layer2( g_seq  )
        h_seq, (h_final,  c_final)     =   self.layer3( h_seq  )

        linear1 = F.relu(self.layer4(h_final))
        score_seq           =   self.layer5( linear1 )
        score_seq = score_seq.squeeze(1) # Unsequeeze the seq dimension
        return score_seq 
        

In [395]:
def initilize_lstm_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    num_classes=2,
                    learning_rate=0.01):
    """Instantiate a RNN model and an optimizer."""

    # Instantiate RNN model
    model = LSTMClassification(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        num_classes=num_classes)
    
    # Send model to `device` (GPU/CPU)
    model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return model, optimizer

In [396]:
set_seed(42)
PATH = "lstm1"
os.makedirs(PATH, exist_ok=True)
model, optimizer = initilize_lstm_model(pretrained_embedding=embeddings.float(),
                                            freeze_embedding=True,
                                            vocab_size=len(word2idx),
                                            embed_dim=300,
                                            num_classes=len(y_info),
                                            learning_rate=0.01)
train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [01:17<00:00, 31.17it/s]


   1    |   2.299730   |  2.291595  |   12.27   |   81.97  


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [01:17<00:00, 31.18it/s]


   2    |   2.293428   |  2.283350  |   13.01   |   81.96  


...

  98    |   0.112009   |  6.059499  |   37.55   |   81.09  


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [01:17<00:00, 31.09it/s]


  99    |   0.109974   |  6.669463  |   34.94   |   81.27  


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [01:17<00:00, 31.12it/s]


  100   |   0.099459   |  6.304997  |   34.94   |   81.20  


Training complete! Best accuracy: 45.35%.


# 4. RNN with attention-based model

In [420]:
class RNNAttentionClassification(nn.Module):

    def __init__(self, vocab_size = None,
                 embed_dim = 128,
                 pretrained_embedding = None,
                 freeze_embedding = True,
                 num_classes = 10):
        super(RNNAttentionClassification, self).__init__()

        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        
        self.encoder = nn.RNN(       embed_dim , embed_dim  , batch_first=True)

        self.linear1 = nn.Linear(embed_dim, embed_dim)
        
        self.adaptivePool1d = nn.AdaptiveAvgPool1d(1)

        self.linear2 = nn.Linear(    embed_dim *2, embed_dim)
        self.linear3 = nn.Linear(    embed_dim, num_classes)

        
    def forward(self, word_seq ):
        embed_out               =   self.embedding( word_seq )
        h_seq, h_final     =   self.encoder( embed_out  )

        vprime = self.linear1(h_final) # Squeeze the len axis to have V'
        alpha = torch.bmm(h_seq, torch.swapaxes(vprime, 1, 2))
        score = torch.nn.functional.softmax(alpha, dim=1) 
        c_w = torch.mul(h_seq, score)
        c_concat = torch.concat([c_w, h_seq], dim=-1) # batch_size, seq, dims
        
        #linear1 = F.relu(self.linear2(c_concat.mean(dim=1)))
        z = F.relu(self.adaptivePool1d(torch.swapaxes(c_concat, 1, 2)))

        score_seq           =   F.relu(self.linear2( z.squeeze(-1) ))
        score_seq = F.relu(self.linear3(score_seq))
        return score_seq 

In [421]:
model = RNNAttentionClassification(len(word2idx), 300, embeddings.float(), True, 10)
from torchinfo import summary
summary(model, [(1,3)], batch_dim=None, dtypes=[torch.int])

Layer (type:depth-idx)                   Output Shape              Param #
RNNAttentionClassification               [1, 10]                   --
├─Embedding: 1-1                         [1, 3, 300]               (12,389,700)
├─RNN: 1-2                               [1, 3, 300]               180,600
├─Linear: 1-3                            [1, 1, 300]               90,300
├─AdaptiveAvgPool1d: 1-4                 [1, 600, 1]               --
├─Linear: 1-5                            [1, 300]                  180,300
├─Linear: 1-6                            [1, 10]                   3,010
Total params: 12,843,910
Trainable params: 454,210
Non-trainable params: 12,389,700
Total mult-adds (M): 13.21
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 51.38
Estimated Total Size (MB): 51.39

In [422]:
def initilize_rnnattention_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    num_classes=2,
                    learning_rate=0.01):
    """Instantiate a RNN model and an optimizer."""

    # Instantiate RNN model
    model = RNNAttentionClassification(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        num_classes=num_classes)
    
    # Send model to `device` (GPU/CPU)
    model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return model, optimizer

In [423]:
# Best accuracy49.07%.
set_seed(42)
rnn_model, optimizer = initilize_rnnattention_model(pretrained_embedding=embeddings.float(),
                                            freeze_embedding=True,
                                            vocab_size=len(word2idx),
                                            embed_dim=300,
                                            num_classes=len(y_info),
                                            learning_rate=0.05)
train(rnn_model, optimizer, train_dataloader, val_dataloader, epochs=50)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:15<00:00, 152.41it/s]


   1    |   2.280712   |  2.194203  |   25.65   |   18.29  


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:16<00:00, 149.26it/s]


   2    |   2.175416   |  2.097893  |   23.05   |   16.90  


...

  48    |   0.967535   |  2.243055  |   43.12   |   15.32  


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:14<00:00, 162.26it/s]


  49    |   0.942749   |  2.255225  |   43.49   |   15.58  


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:15<00:00, 156.51it/s]


  50    |   0.939520   |  2.300923  |   42.38   |   16.14  


Training complete! Best accuracy: 49.07%.


# 5. LSTM with Attention



In [19]:
class LSTMAttentionClassification(nn.Module):

    def __init__(self, vocab_size = None,
                 embed_dim = 128,
                 pretrained_embedding = None,
                 freeze_embedding = True,
                 num_classes = 10):
        super(LSTMAttentionClassification, self).__init__()

        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        
        self.encoder = nn.LSTM(       embed_dim , embed_dim  , batch_first=True)

        self.linear1 = nn.Linear(embed_dim, embed_dim)
        
        self.adaptivePool1d = nn.AdaptiveAvgPool1d(1)

        self.linear2 = nn.Linear(    embed_dim *2, embed_dim)
        self.linear3 = nn.Linear(    embed_dim, num_classes)

        
    def forward(self, word_seq ):
        embed_out               =   self.embedding( word_seq )
        h_seq, (h_final, c_final)     =   self.encoder( embed_out  )

        vprime = self.linear1(h_final) # Squeeze the len axis to have V'
        alpha = torch.bmm(h_seq, torch.swapaxes(vprime, 1, 2))
        score = torch.nn.functional.softmax(alpha, dim=1) 
        c_w = torch.mul(h_seq, score)
        c_concat = torch.concat([c_w, h_seq], dim=-1) # batch_size, seq, dims
        
        #linear1 = F.relu(self.linear2(c_concat.mean(dim=1)))
        z = F.relu(self.adaptivePool1d(torch.swapaxes(c_concat, 1, 2)))

        score_seq           =   F.relu(self.linear2( z.squeeze(-1) ))
        score_seq = self.linear3(score_seq)
        return score_seq 

In [20]:
def initilize_lstmattention_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    num_classes=2,
                    learning_rate=0.01):
    """Instantiate a RNN model and an optimizer."""

    # Instantiate RNN model
    model = LSTMAttentionClassification(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        num_classes=num_classes)
    
    # Send model to `device` (GPU/CPU)
    model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return model, optimizer

In [21]:
# 49.07%.
PATH = 'lstmattention'
os.makedirs(PATH, exist_ok=True)
set_seed(42)
model, optimizer = initilize_lstmattention_model(pretrained_embedding=embeddings.float(),
                                            freeze_embedding=True,
                                            vocab_size=len(word2idx),
                                            embed_dim=300,
                                            num_classes=len(y_info),
                                            learning_rate=0.1)
train(model, optimizer, train_dataloader, val_dataloader, epochs=50)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:24<00:00, 97.08it/s]


   1    |   2.284548   |  2.248345  |   13.01   |   26.78  


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:24<00:00, 97.82it/s]


   2    |   2.119614   |  1.907035  |   30.86   |   26.49  


...

  48    |   0.021439   |  7.743779  |   47.21   |   25.68  


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:24<00:00, 98.52it/s]


  49    |   0.008540   |  8.200143  |   44.61   |   25.70  


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:24<00:00, 98.65it/s]


  50    |   0.005210   |  8.062489  |   46.10   |   25.68  


Training complete! Best accuracy: 55.76%.


# 6. Transformer

In [435]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = nn.MultiheadAttention(num_heads=num_heads, embed_dim=embed_dim, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim ,embed_dim)
        )
        self.layernorm1 = nn.LayerNorm(normalized_shape = [embed_dim], eps=1e-6)
        self.layernorm2 = nn.LayerNorm(normalized_shape = [embed_dim], eps=1e-6)
        self.dropout1 = nn.Dropout(rate)
        self.dropout2 = nn.Dropout(rate)

    def forward(self, inputs):
        attn_output, _ = self.att(inputs, inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

In [444]:
class TokenAndPositionEmbedding(nn.Module):
    def __init__(self, maxlen, vocab_size, embed_dim, pretrained_embedding, freeze_embedding):
        super(TokenAndPositionEmbedding, self).__init__()

        if pretrained_embedding is not None:
            self.token_emb = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.token_emb = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)


        self.pos_emb = nn.Embedding(num_embeddings=maxlen, embedding_dim=embed_dim)

    def forward(self, x):
        maxlen = x.size()[-1]
        positions = torch.arange(start=0, end=maxlen, step=1).to(device)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [460]:
class TransformerClassification(nn.Module):
  def __init__(self, maxlen, vocab_size, embed_dim, pretrained_embedding, freeze_embedding, num_heads, ff_dim, num_classes):
    super(TransformerClassification, self).__init__()

    self.embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim, pretrained_embedding, freeze_embedding)
    self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    self.averagePool = nn.AdaptiveAvgPool1d(output_size=1)
    self.sequential = nn.Sequential(
        nn.Dropout(0.1),
        nn.Linear(embed_dim, 20),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(20, num_classes)
    )
    

  def forward(self, x):
    x = self.embedding_layer(x)
    #print(x.size())
    x = self.transformer_block(x)
    #print(x.size())

    # Swap from (N, L, features) to (N, features, L)
    x = torch.swapaxes(x, 1, 2)
    x = self.averagePool(x)
    x = x.squeeze(-1)
    #print(x.size())

    x = self.sequential(x)
    return x



In [461]:
#!pip install torchinfo
model = TransformerClassification(pretrained_embedding=embeddings.float(),
                        freeze_embedding=True,
                        vocab_size=len(word2idx),
                        embed_dim=300,
                        maxlen = max_len,
                        num_classes=10,
                        num_heads = 2,
                        ff_dim = 300)
from torchinfo import summary
summary(model.to(device), [(1,100)], batch_dim=None, dtypes=[torch.int])

Layer (type:depth-idx)                   Output Shape              Param #
TransformerClassification                [1, 10]                   --
├─TokenAndPositionEmbedding: 1-1         [1, 100, 300]             --
│    └─Embedding: 2-1                    [100, 300]                696,300
│    └─Embedding: 2-2                    [1, 100, 300]             (12,389,700)
├─TransformerBlock: 1-2                  [1, 100, 300]             --
│    └─MultiheadAttention: 2-3           [1, 100, 300]             361,200
│    └─Dropout: 2-4                      [1, 100, 300]             --
│    └─LayerNorm: 2-5                    [1, 100, 300]             600
│    └─Sequential: 2-6                   [1, 100, 300]             --
│    │    └─Linear: 3-1                  [1, 100, 300]             90,300
│    │    └─ReLU: 3-2                    [1, 100, 300]             --
│    │    └─Linear: 3-3                  [1, 100, 300]             90,300
│    └─Dropout: 2-7                      [1, 100, 300]  

In [462]:
def initilize_transformer_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim = 300,
                    num_classes=2,
                    maxlen = 10000,
                    learning_rate=0.01,
                    num_heads = 2,
                    ff_dim = 300
                    ):
    """Instantiate a RNN model and an optimizer."""

    # Instantiate transformer model
    model = TransformerClassification(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        maxlen = maxlen,
                        num_classes=num_classes,
                        num_heads = num_heads,
                        ff_dim = ff_dim)
    
    # Send model to `device` (GPU/CPU)
    model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return model, optimizer

In [463]:
# 44.98% accuracy
set_seed(42)
model, optimizer = initilize_transformer_model(pretrained_embedding=embeddings.float(),
                                            freeze_embedding=False,
                                            vocab_size=len(word2idx),
                                            embed_dim=300,
                                            num_classes=len(y_info),
                                            maxlen = max_len,
                                            learning_rate=0.1,
                                            num_heads = 10,
                                            ff_dim = 300)
train(model, optimizer, train_dataloader, val_dataloader, epochs=100)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:16<00:00, 145.66it/s]


   1    |   2.293620   |  2.269196  |   17.10   |   19.05  


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:16<00:00, 144.66it/s]


   2    |   2.168927   |  1.988105  |   29.00   |   18.97  


...

  98    |   0.009990   | 10.893432  |   39.41   |   16.97  


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:16<00:00, 143.25it/s]


  99    |   0.006492   | 10.291065  |   39.41   |   16.99  


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2413/2413 [00:17<00:00, 141.71it/s]


  100   |   0.005614   | 10.667309  |   40.15   |   17.17  


Training complete! Best accuracy: 44.98%.
